<a href="https://colab.research.google.com/github/ngnadeera/tensorflow_model/blob/main/Tensorflow_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

# Load dataset.
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
# print(dftrain.head()) Get the whole data frame
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

# print(dftrain.head())  data frame without the survived column
# print(y_train)  Only the survive column
# print (dftrain.loc[2])  each specific row
# print (dftrain["age"])  each specific column
# print(dftrain.describe())  some insigts on the dataset
# print(dftrain.shape) (629,9)




In [21]:

CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = []

# For the categorical columns

for feature_name in CATEGORICAL_COLUMNS:

  vocabulary = dftrain[feature_name].unique()  # gets a list of all unique values from given feature column

  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

# For the Numerical Columns

for feature_name in NUMERIC_COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))



In [35]:
# We load data into the model in batches INPUT FUNCTION

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():  # inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)



In [46]:

# We create a linear estimtor by passing the feature columns we created earlier
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)

linear_est.train(train_input_fn)  # train
result = linear_est.evaluate(eval_input_fn)  # get model metrics/stats by testing on tetsing data

clear_output()  # clears consoke output
print(result['accuracy'])  # the result variable is simply a dict of stats about our model



0.7386364


In [49]:
result = list(linear_est.predict(eval_input_fn))
print(dfeval.loc[10])
print(y_eval.loc[10])
print(result[10]['probabilities'][1])


sex                       female
age                         28.0
n_siblings_spouses             0
parch                          0
fare                        7.75
class                      Third
deck                     unknown
embark_town           Queenstown
alone                          y
Name: 10, dtype: object
1
0.73100024
